In [110]:
# %pip install --upgrade xlrd
# %pip install pyarrow
# %pip install openpyxl
# %pip install yahoo_fin
# %pip install alpha_vantage
# %pip install yfinance

import pandas as pd
import requests
import time
from datetime import datetime, date, timedelta
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

# import yfinance as yf
# from yahoo_fin.stock_info import get_data
# from alpha_vantage.timeseries import TimeSeries

In [111]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
start_date = datetime(year=1967, month=1, day=1)
now = datetime.now()

In [112]:
# https://fred.stlouisfed.org/series/FEDFUNDS

df = pd.read_excel('https://fred.stlouisfed.org/graph/fredgraph.xls?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=FEDFUNDS&scale=left&cosd=1954-07-01&coed=2022-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-07-06&revision_date=2022-07-06&nd=1954-07-01',
                    header=10, 
                    names=['date','FEDrate'])

df = df[df['date'] >= start_date]
df['year'] = df['date'].apply(lambda x: x.year)
df['month'] = df['date'].apply(lambda x: x.month)
df['day'] = df['date'].apply(lambda x: x.day)

df = df.sort_values(by="date")
df = df.reset_index(drop=True)
df.to_feather('Data/FEDFUNDS.ftr')

In [113]:
# https://fred.stlouisfed.org/series/EFFRVOL

df = pd.read_excel('https://fred.stlouisfed.org/graph/fredgraph.xls?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=EFFRVOL&scale=left&cosd=2017-07-05&coed=2022-07-05&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-07-06&revision_date=2022-07-06&nd=2016-03-01', 
                    header=10, 
                    names=['date', 'EFFRVol'])

df = df[df['date'] >= start_date]
df['year'] = df['date'].apply(lambda x: x.year)
df['month'] = df['date'].apply(lambda x: x.month)
df['day'] = df['date'].apply(lambda x: x.day)

df = df.sort_values(by="date")
df = df.reset_index(drop=True)
df.to_feather('Data/EFFRVOL.ftr')

In [123]:
df = pd.read_excel('Data/SPX.xlsx')
df['Date'] = df['Date'].apply(parse)
df.columns = ['SPX-' + col.lower().replace('*','') for col in df.columns]
df = df.rename(columns={'SPX-date':'date', 'SPX-volume':'volume'})

df = df[df['date'] >= start_date]
df['year'] = df['date'].apply(lambda x: x.year)
df['month'] = df['date'].apply(lambda x: x.month)
df['day'] = df['date'].apply(lambda x: x.day)

df = df.sort_values(by="date")
df = df.reset_index(drop=True)
df.to_feather('Data/SPX.ftr')

In [115]:
# https://download.bls.gov/pub/time.series/cu/cu.txt

CPI_types = ['11.USFoodBeverage', '12.USHousing', '13.USApparel', '14.USTransportation', '15.USMedical', '16.USRecreation', '17.USEducationAndCommunication', '18.USOtherGoodsAndServices', '20.USCommoditiesServicesSpecial']
CPI_dict = {}

for name in CPI_types:
    df = pd.read_csv(f'https://download.bls.gov/pub/time.series/cu/cu.data.{name}', delimiter='\t')
    df = df.iloc[:,:4]
    df.columns = ['series_id','year','month','CPI']
    df['month'] = df['month'].apply(lambda x: int(x[1:]))
    df = df[df['month'] != 13]
    df = df.reset_index(drop=True)

    # Calculating the mean CPI for each of the categories in CPI_types,
    # and filling a dictionary with those series.
    new_CPI = []
    for y in df['year'].unique():
        for m in df['month'].unique():
            year = df[df['year']==y]
            month = year[year['month']==m]
            thingy = (y, m, month['CPI'].mean())
            new_CPI.append(thingy)
    new_name = name.split('.')[1]
    CPI_dict[f'{name}'] = pd.DataFrame(new_CPI, columns=['year', 'month', f'CPI_{new_name}'])


df = CPI_dict[list(CPI_dict.keys())[0]][['year', 'month']]
for key in list(CPI_dict.keys()):
    df[key] = CPI_dict[key].iloc[:,2]

df['day'] = 1
date_creation = df[['year', 'month', 'day']].astype(str).copy()
date_creation['date'] = date_creation['year']+'-'+date_creation['month']+'-'+date_creation['day']
df['date'] = date_creation['date'].apply(parse)
df = df[df['date'] >= start_date]
df = df.reindex(columns=['date', 'year','month','day','11.USFoodBeverage', '12.USHousing', '13.USApparel', '14.USTransportation', '15.USMedical', '16.USRecreation', '17.USEducationAndCommunication', '18.USOtherGoodsAndServices', '20.USCommoditiesServicesSpecial'])

df = df.reset_index(drop=True)
df.to_feather('Data/CPI.ftr')

In [125]:
a = pd.read_feather('Data/FEDFUNDS.ftr')
b = pd.read_feather('Data/EFFRVOL.ftr')
c = pd.read_feather('Data/SPX.ftr')
d = pd.read_feather('Data/CPI.ftr')

# Merging datasets, filling in values of missing dates using the previous value.
joined = pd.merge_asof(c, a, on='date', by=['year', 'month', 'day'])
joined = pd.merge_asof(joined, b, on='date', by=['year', 'month', 'day'])
joined = pd.merge_asof(joined, d, on='date', by=['year', 'month', 'day'])
#joined = pd.merge_asof(joined, pd.DataFrame({"date":joined.date, "SPX-7":joined['SPX'].rolling(window=7).mean()}).reset_index(drop=True), on='date')
joined = joined.reset_index(drop=True)

# # There were some places where EFFRVol = 0, so I filled them with the mean of preceding and following values
# for i in range(len(joined)):
#     if joined['EFFRVol'][i] == 0:
#         joined['EFFRVol'][i] = (joined['EFFRVol'][i-1] + joined['EFFRVol'][i+1]) / 2

joined = joined.fillna(method='ffill')
joined = joined.fillna(method='bfill')

# Creating a column to signal when (and how) the Fed changes rate
FEDrate_delta = []
for i in range(len(joined) - 1):
    FEDrate_delta.append(joined['FEDrate'][i+1] - joined['FEDrate'][i])
FEDrate_delta.append(0)
joined['FEDrate_delta'] = FEDrate_delta

joined.to_feather('Data/cleaned_dataset.ftr')

In [126]:
joined.head()

,date,SPX-open,SPX-high,SPX-low,SPX-close,SPX-adj close,volume,year,month,day,FEDrate,EFFRVol,11.USFoodBeverage,12.USHousing,13.USApparel,14.USTransportation,15.USMedical,16.USRecreation,17.USEducationAndCommunication,18.USOtherGoodsAndServices,20.USCommoditiesServicesSpecial,FEDrate_delta
0,1967-01-03,0.0,81.61,79.59,80.38,80.38,6100000,1967,1,3,5.0,83.0,35.450704,26.56,42.290909,16.642857,13.3,127.320833,134.095833,36.25,90.78,0.0
1,1967-01-04,0.0,81.01,79.43,80.55,80.55,6150000,1967,1,4,5.0,83.0,35.450704,26.56,42.290909,16.642857,13.3,127.320833,134.095833,36.25,90.78,0.0
2,1967-01-05,0.0,81.93,80.50,81.60,81.60,7320000,1967,1,5,5.0,83.0,35.450704,26.56,42.290909,16.642857,13.3,127.320833,134.095833,36.25,90.78,0.0
3,1967-01-06,0.0,82.79,81.32,82.18,82.18,7830000,1967,1,6,5.0,83.0,35.450704,26.56,42.290909,16.642857,13.3,127.320833,134.095833,36.25,90.78,0.0
4,1967-01-09,0.0,83.31,81.78,82.81,82.81,9180000,1967,1,9,5.0,83.0,35.450704,26.56,42.290909,16.642857,13.3,127.320833,134.095833,36.25,90.78,0.0
